#Задача:
Подготовить данные для модели:
    
   Анализ данных, необходимые корректировки. 
1. Обработать пропуски. 
2. Оценить выбросы. 
3. Корреляция. 
4. Тест на нормальность распределения.
5. Масштабировать данные.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Задаем некоторые опции библиотеки pandas, которые 
# настраивают вывод
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 80)

In [93]:
#df = pd.read_csv("./credit_train.csv", encoding='cp1251', on_bad_lines = 'skip', sep=';')
# Прочитаем файл данных
df = pd.read_csv("./credit_train.csv", encoding='ANSI', on_bad_lines = 'skip', sep=';')
df


,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,NaN,NaN,UMN,"59998,00",10,1.60,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,NaN,MAR,UMN,"10889,00",6,1.10,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,3,M,32.0,MAR,SPC,"10728,00",12,1.10,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,4,F,27.0,NaN,SPC,"12009,09",12,1.10,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,5,M,45.0,NaN,SPC,NaN,10,1.10,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170741,170742,F,27.0,UNM,SPC,"64867,00",12,1.10,"0,535257",GRD,РЕСПУБЛИКА ТАТАРСТАН,40000.0,6.0,0.0,0
170742,170743,F,24.0,MAR,SPC,"17640,00",6,1.60,"0,573287",SCH,САНКТ-ПЕТЕРБУРГ Г,30000.0,1.0,0.0,0
170743,170744,F,31.0,UNM,SPC,"27556,47",10,1.32,"0,416098",GRD,ПРИМОРСКИЙ КРАЙ,40000.0,1.0,0.0,0
170744,170745,F,53.0,DIV,PNA,"6189,00",12,1.10,"0,482595",SCH,ПЕНЗЕНСКАЯ ОБЛ,31000.0,2.0,0.0,0


In [38]:
# Изменим тип графы living_region на str для дольнейшей обработки данных
df["living_region"] = df["living_region"].astype("str")
df.dtypes

client_id                 int64
gender                   object
age                     float64
marital_status           object
job_position             object
                         ...   
living_region            object
monthly_income          float64
credit_count            float64
overdue_credit_count    float64
open_account_flg          int64
Length: 15, dtype: object

In [37]:
# Напишем метод очистки данных от мусорных слов и символов
def clear_data(val):
          new_val = val.replace(" Г", "").replace(".", "").replace("/", "").replace("(", "").replace(")", "")
          new_val = new_val.replace("АВТОНОМНЫЙ ОКРУГ - ЮГРА", "").replace("АВТОНОМНЫЙ ОКРУГ - Ю", "").replace("АО", "").replace("РЕСПУБЛИКА", "").replace("РЕСП", "").replace("ОБЛАСТЬ", "").replace("ОБЛ", "").replace("КРАЙ", "").replace("САХА", "")
          new_val = new_val.replace(" ", "")
          return str(new_val)

In [39]:
# Применим метод очистки данных ко всей графе living_region
df["living_region"] = df["living_region"].apply(clear_data)
df

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,NaN,NaN,UMN,"59998,00",10,1.60,NaN,GRD,КРАСНОДАРСКИЙ,30000.0,1.0,1.0,0
1,2,F,NaN,MAR,UMN,"10889,00",6,1.10,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,3,M,32.0,MAR,SPC,"10728,00",12,1.10,NaN,NaN,САРАТОВСКАЯ,NaN,5.0,0.0,0
3,4,F,27.0,NaN,SPC,"12009,09",12,1.10,NaN,NaN,ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,5,M,45.0,NaN,SPC,NaN,10,1.10,"0,421385",SCH,ЧЕЛЯБИНСКАЯ,NaN,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170741,170742,F,27.0,UNM,SPC,"64867,00",12,1.10,"0,535257",GRD,ТАТАРСТАН,40000.0,6.0,0.0,0
170742,170743,F,24.0,MAR,SPC,"17640,00",6,1.60,"0,573287",SCH,САНКТ-ПЕТЕРБУРГ,30000.0,1.0,0.0,0
170743,170744,F,31.0,UNM,SPC,"27556,47",10,1.32,"0,416098",GRD,ПРИМОРСКИЙ,40000.0,1.0,0.0,0
170744,170745,F,53.0,DIV,PNA,"6189,00",12,1.10,"0,482595",SCH,ПЕНЗЕНСКАЯ,31000.0,2.0,0.0,0


In [40]:
# Заменим буквы пола в столбце gender на цифры 0 и 1(M = 0, F = 1)
df["gender"] = np.where(df["gender"] == "M", 0, 1)
df

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,0,NaN,NaN,UMN,"59998,00",10,1.60,NaN,GRD,КРАСНОДАРСКИЙ,30000.0,1.0,1.0,0
1,2,1,NaN,MAR,UMN,"10889,00",6,1.10,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,3,0,32.0,MAR,SPC,"10728,00",12,1.10,NaN,NaN,САРАТОВСКАЯ,NaN,5.0,0.0,0
3,4,1,27.0,NaN,SPC,"12009,09",12,1.10,NaN,NaN,ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,5,0,45.0,NaN,SPC,NaN,10,1.10,"0,421385",SCH,ЧЕЛЯБИНСКАЯ,NaN,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170741,170742,1,27.0,UNM,SPC,"64867,00",12,1.10,"0,535257",GRD,ТАТАРСТАН,40000.0,6.0,0.0,0
170742,170743,1,24.0,MAR,SPC,"17640,00",6,1.60,"0,573287",SCH,САНКТ-ПЕТЕРБУРГ,30000.0,1.0,0.0,0
170743,170744,1,31.0,UNM,SPC,"27556,47",10,1.32,"0,416098",GRD,ПРИМОРСКИЙ,40000.0,1.0,0.0,0
170744,170745,1,53.0,DIV,PNA,"6189,00",12,1.10,"0,482595",SCH,ПЕНЗЕНСКАЯ,31000.0,2.0,0.0,0


In [60]:
# Удалим пропуски в графе
df = df.dropna()
df

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
7,8,F,26.0,UNM,13.0,"47878,00",10,1.10,"0,512525",GRD,МОСКОВСКАЯ ОБЛ,60000.0,3.0,0.0,0
9,10,F,32.0,UNM,13.0,"26268,00",10,1.10,"0,465026",GRD,КРАЙ СТАВРОПОЛЬСКИЙ,39500.0,7.0,0.0,0
10,11,M,26.0,MAR,13.0,"16793,00",14,1.00,"0,445430",SCH,САНКТ-ПЕТЕРБУРГ,36000.0,2.0,0.0,0
12,13,M,37.0,MAR,13.0,"42289,00",10,1.60,"0,691609",SCH,ОБЛ АРХАНГЕЛЬСКАЯ,70000.0,1.0,0.0,0
13,14,M,43.0,MAR,5.0,"57567,00",10,1.10,"0,341164",GRD,ХАНТЫ-МАНСИЙСКИЙ АО,60000.0,7.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170741,170742,F,27.0,UNM,13.0,"64867,00",12,1.10,"0,535257",GRD,РЕСПУБЛИКА ТАТАРСТАН,40000.0,6.0,0.0,0
170742,170743,F,24.0,MAR,13.0,"17640,00",6,1.60,"0,573287",SCH,САНКТ-ПЕТЕРБУРГ Г,30000.0,1.0,0.0,0
170743,170744,F,31.0,UNM,13.0,"27556,47",10,1.32,"0,416098",GRD,ПРИМОРСКИЙ КРАЙ,40000.0,1.0,0.0,0
170744,170745,F,53.0,DIV,9.0,"6189,00",12,1.10,"0,482595",SCH,ПЕНЗЕНСКАЯ ОБЛ,31000.0,2.0,0.0,0


In [95]:
# Заменим буквенные статусы в графее marital_status на цифры 0 и 1 и -1 (UNM = 0, MAR = 1, DIV = -1)
df["marital_status"] = np.where(df["marital_status"] == "UNM", 0, (np.where(df["marital_status"] == "MAR", 1, -1)))
df

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,NaN,-1,UMN,"59998,00",10,1.60,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,NaN,1,UMN,"10889,00",6,1.10,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,3,M,32.0,1,SPC,"10728,00",12,1.10,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,0
3,4,F,27.0,-1,SPC,"12009,09",12,1.10,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,5,M,45.0,-1,SPC,NaN,10,1.10,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170741,170742,F,27.0,0,SPC,"64867,00",12,1.10,"0,535257",GRD,РЕСПУБЛИКА ТАТАРСТАН,40000.0,6.0,0.0,0
170742,170743,F,24.0,1,SPC,"17640,00",6,1.60,"0,573287",SCH,САНКТ-ПЕТЕРБУРГ Г,30000.0,1.0,0.0,0
170743,170744,F,31.0,0,SPC,"27556,47",10,1.32,"0,416098",GRD,ПРИМОРСКИЙ КРАЙ,40000.0,1.0,0.0,0
170744,170745,F,53.0,-1,PNA,"6189,00",12,1.10,"0,482595",SCH,ПЕНЗЕНСКАЯ ОБЛ,31000.0,2.0,0.0,0


In [97]:
# Заменим буквенные статусы в графее marital_status на цифры 0 и 1 и -1 (UNM = 0, MAR = 1, DIV = -1)
#print(np.where(df["job_position"] == "SPC", 0, ))
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
job_position = ord_enc.fit_transform(df[["job_position"]])
job_position


array([[14.],
       [14.],
       [13.],
       ...,
       [13.],
       [ 9.],
       [13.]])

In [80]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
one_hot_encoded = encoder.fit_transform(df[["living_region"]])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(["living_region"]))
one_hot_df

ValueError: Shape of passed values is (161331, 1), indices imply (161331, 298)